## Speed Dating dataset analysis
In this notebook, our goal is to analyze all the scikit learn pipelines that are executed on the task 146607 (https://www.openml.org/t/146607) which is a supervised classficaiton task using the speed dating dataset (https://www.openml.org/d/40536).
First, we find all the (scikit learn) pipelines and their specific runs (hyper-parameter settings).
Using these pipelines and runs, we populate the database.
Then we the pipeline graph.
Using the pipeline graph, we show how we can speed up the process of pipeline execution and hyperparamter search.

After finding all the "good" pipelines, there are two strategies that we can use for the experiment:
- Order all the runs based on their timestamp, start from an empty database
- Divide the pipelines into two groups, populate the databse with one group, and use the other group to execute the workload (incrementally)



In [1]:
from openml import datasets, tasks, runs, flows, setups, config
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sys

config.apikey = '8e2079508005379adb9a60798696134e'
config.server = 'https://www.openml.org/api/v1'
config.set_cache_directory(os.path.expanduser('~/openml-cache'))